In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler,PowerTransformer,MinMaxScaler,QuantileTransformer

from sklearn.feature_selection import VarianceThreshold, SelectKBest

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import normalize

# Load the first database Amazon

In [19]:
df = pd.read_csv('Datasets/amazon/amazon_review_ID.shuf.lrn.csv')
df.head()

X_test = pd.read_csv('Datasets/amazon/amazon_review_ID.shuf.tes.csv').drop('ID', axis=1)
X_test
df


,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V9992,V9993,V9994,V9995,V9996,V9997,V9998,V9999,V10000,Class
0,0,8,6,10,6,7,2,2,2,3,...,1,0,0,0,0,0,0,0,0,Chell
1,1,13,1,13,9,8,8,2,3,2,...,4,1,2,1,0,1,0,0,0,Engineer
2,2,16,7,6,7,9,3,4,2,6,...,0,0,0,0,1,0,0,0,0,Grove
3,3,8,11,10,11,3,7,0,4,2,...,0,0,1,0,0,0,2,1,0,Davisson
4,4,10,11,8,5,3,4,2,5,5,...,0,0,0,0,0,0,0,0,0,Wilson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,745,9,8,5,5,6,2,14,1,4,...,0,1,0,0,1,0,1,0,0,Riley
746,746,6,13,4,6,8,2,7,3,3,...,0,0,0,0,1,0,0,0,1,Neal
747,747,16,11,4,9,9,4,0,3,5,...,0,0,0,0,0,1,1,0,4,Messick
748,748,9,5,4,2,3,7,2,7,1,...,0,0,0,0,0,2,0,0,0,Mitchell


# Change the attributes from the Class coloum into numbers 

In [20]:
Class_description = []
Class_description = pd.unique(df.Class)

for i in range(len(Class_description)):
    df.Class[df.Class == Class_description[i]] = i+1

In [37]:
X_lrn = df.loc[:,'V1':'V10000']
Y_lrn = df.loc[:,'Class':'Class'].astype(float)

In [22]:
def MLP_Search(alphas,modes,solv, h,maxiter,X_train, X_valid, Y_train, Y_valid):
    erg = []
    for g in h:
        for i in alphas:
            for j in modes:
                for k in solv:
                    clf = MLPClassifier(hidden_layer_sizes=(g), max_iter=maxiter, alpha=i,solver=k,activation=j,tol = 1e-9)
                    clf.fit(X_train, Y_train)
                    Y_pred = clf.predict(X_valid)
                    print("Hidden layers: ",g,"|\talpha: ",i,"|\tmode: ",j,"|\tsolver: ",k,"|\tscore: ",accuracy_score(Y_valid, Y_pred))
                    erg.append({
                        "h": g,
                        "alpha": i,
                        "mode": j,
                        "solver": k,
                        "score": accuracy_score(Y_valid, Y_pred)})
        print("")
    return erg

In [23]:
def Statistic(Y_valid,Y_pred,name):
    print("Heat map: ")
    plt.figure()
    cm = confusion_matrix(Y_valid.Class, Y_pred)
    sns.heatmap(cm, center=True)
    plt.savefig("Heatmap {}".format(name))
    plt.figure()
    sns.distplot(Y_valid.Class)
    sns.distplot(Y_pred, color="red")
    plt.savefig("difference between prediction and validation {}".format(name))

    plt.figure()
    sns.distplot(Y_valid.Class-Y_pred)
    plt.savefig("total difference between prediction and validation {}".format(name))
    print(sqrt(mean_squared_error(Y_valid.Class, Y_pred)))

    Y_pred_Norm = Y_pred / np.linalg.norm(Y_pred)
    Y_valid_Norm = Y_valid / np.linalg.norm(Y_valid.Class)

    print(sqrt(mean_squared_error(Y_pred_Norm, Y_valid_Norm)))

In [24]:
def FindBestScore(results):
    best_score = 0
    best_index = 0
    for i, res in enumerate(results):
        if res["score"] > best_score:
            best_score = res["score"]
            best_index = i
    print("best_score:", best_score)
    print("best_params:", results[best_index])
    return results[best_index]

In [25]:
def plotLC(LC,best_params):
    plt.figure()
    plt.plot(LC)#,label="activation {} Hidden Layers {} solver {} ".format(best_params["alpha"],i,l))
    plt.grid()
    plt.legend()
    plt.xlabel("Iterations")
    plt.xlabel("Loss")
    plt.title("activation {} Hidden Layers {} solver {} ".format(best_params["mode"],best_params["h"],best_params["solver"]))
    plt.savefig("activation {} Hidden Layers {} solver {} ".format(best_params["mode"],best_params["h"],best_params["solver"]))

# Feature Selection with K-Best

In [26]:
from math import sqrt

#VT = VarianceThreshold(threshold=0.05)
features = df.loc[:,'V1':'V10000']
target = df.loc[:,'Class']
num_features = features.shape[1]
features

# Create and fit selector
k= int(sqrt(num_features)*100)
print(k)
selector = SelectKBest(k=k)
# Get columns to keep and create new dataframe with those only
cols = selector.fit(features.values, target.values).get_support(indices=True)
selected_feats = features.iloc[:,cols]
selected_feats

10000


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V9991,V9992,V9993,V9994,V9995,V9996,V9997,V9998,V9999,V10000
0,8,6,10,6,7,2,2,2,3,2,...,0,1,0,0,0,0,0,0,0,0
1,13,1,13,9,8,8,2,3,2,4,...,0,4,1,2,1,0,1,0,0,0
2,16,7,6,7,9,3,4,2,6,5,...,0,0,0,0,0,1,0,0,0,0
3,8,11,10,11,3,7,0,4,2,1,...,2,0,0,1,0,0,0,2,1,0
4,10,11,8,5,3,4,2,5,5,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,9,8,5,5,6,2,14,1,4,5,...,0,0,1,0,0,1,0,1,0,0
746,6,13,4,6,8,2,7,3,3,2,...,0,0,0,0,0,1,0,0,0,1
747,16,11,4,9,9,4,0,3,5,0,...,0,0,0,0,0,0,1,1,0,4
748,9,5,4,2,3,7,2,7,1,1,...,0,0,0,0,0,0,2,0,0,0


In [27]:
features

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V9991,V9992,V9993,V9994,V9995,V9996,V9997,V9998,V9999,V10000
0,8,6,10,6,7,2,2,2,3,2,...,0,1,0,0,0,0,0,0,0,0
1,13,1,13,9,8,8,2,3,2,4,...,0,4,1,2,1,0,1,0,0,0
2,16,7,6,7,9,3,4,2,6,5,...,0,0,0,0,0,1,0,0,0,0
3,8,11,10,11,3,7,0,4,2,1,...,2,0,0,1,0,0,0,2,1,0
4,10,11,8,5,3,4,2,5,5,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,9,8,5,5,6,2,14,1,4,5,...,0,0,1,0,0,1,0,1,0,0
746,6,13,4,6,8,2,7,3,3,2,...,0,0,0,0,0,1,0,0,0,1
747,16,11,4,9,9,4,0,3,5,0,...,0,0,0,0,0,0,1,1,0,4
748,9,5,4,2,3,7,2,7,1,1,...,0,0,0,0,0,0,2,0,0,0


# Set Features:

In [28]:
alphas = [1e-3] 
activation = ["relu","identity","logistic","tanh"]
solver = ["sgd","adam"]
learning_rate = ["constant"]
h = [100,300,600,900]

alphas = [1e-4]
activation = ["relu"]
solver = ["adam"]
h = [300]
maxiter = 1000

# Standart Scaler

In [39]:
X_train, X_valid, Y_train, Y_valid = train_test_split(selected_feats, Y_lrn, test_size=0.4)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
scaler.fit(X_train) 
X_trainSC = scaler.transform(X_train)
X_validSC = scaler.transform(X_valid) 
print(X_validSC.shape)

(300, 10000)


In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
scaler.fit(X_lrn) 
X_trainSC = scaler.transform(X_lrn)
X_validSC = scaler.transform(X_valid) 

In [40]:
#Best score relu adam hidden layer = 300 Iter 1000 learning rate = constant score=0.66

results = MLP_Search(alphas,activation,solver,h,maxiter,X_trainSC, X_validSC, Y_train, Y_valid)
best_params_SC = FindBestScore(results)

In [ ]:
#Best score relu adam hidden layer = 300 Iter 1000 learning rate = constant score=0.66 
clf = MLPClassifier(hidden_layer_sizes=(best_params_SC["h"]), max_iter=maxiter, alpha=best_params_SC["alpha"],solver=best_params_SC["solver"],activation=best_params_SC["mode"],tol=1e-9,verbose=False)
clf.fit(X_trainSC, Y_train)
Y_predSC = clf.predict(selected_feats)
LC = clf.loss_curve_

print(accuracy_score(Y_valid, Y_predSC))
#Statistic(Y_valid,Y_predSC,"MLP SC")
#plotLC(LC,best_params_SC)

In [ ]:
print(Y_predSC.shape)

# Normalize

In [ ]:
from sklearn.preprocessing import normalize
X_trainN = normalize(X_train, norm='l2')
X_validN = normalize(X_valid, norm='l2')

In [ ]:
results = MLP_Search(alphas,activation,solver,h,maxiter,X_trainN, X_validN, Y_train, Y_valid)
best_params_N = FindBestScore(results)

# alpha 0.001 tanh adam learning rate = constant hidden layer 900 Inter 1000 score 0.47

In [ ]:
#Best score relu adam hidden layer = 300 Iter 1000 learning rate = constant score=0.66 
clf = MLPClassifier(hidden_layer_sizes=(best_params_N["h"]), max_iter=maxiter, alpha=best_params_N["alpha"],solver=best_params_N["solver"],activation=best_params_N["mode"],tol=1e-9,verbose=False)
clf.fit(X_trainN, Y_train)
Y_predN = clf.predict(X_validN)
LC = clf.loss_curve_
Statistic(Y_valid,Y_predN,"MLP N")
plotLC(LC,best_params_N)

# MinMaxscaler

In [ ]:
min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_Valid_minmax = min_max_scaler.fit_transform(X_valid)

In [ ]:
results = MLP_Search(alphas,activation,solver,h,maxiter,X_train_minmax, X_Valid_minmax, Y_train, Y_valid)
best_params_minmax = FindBestScore(results)
# alpha 0.001 tanh adam learning rate = constant hidden layer 900 Inter 1000 score 0.61

In [ ]:
#Best score relu adam hidden layer = 300 Iter 1000 learning rate = constant score=0.66 
clf = MLPClassifier(hidden_layer_sizes=(best_params_minmax["h"]), max_iter=maxiter, alpha=best_params_minmax["alpha"],solver=best_params_minmax["solver"],activation=best_params_minmax["mode"],tol=1e-9,verbose=False)
clf.fit(X_trainN, Y_train)
Y_pred_minmax = clf.predict(X_Valid_minmax)
LC = clf.loss_curve_
Statistic(Y_valid,Y_pred_minmax,"MLP minmax")
plotLC(LC,best_params_minmax)

# quantile_transformer

In [ ]:
quantile_transformer = QuantileTransformer(random_state=0)
X_train_trans = quantile_transformer.fit_transform(X_train)
X_Valid_trans = quantile_transformer.transform(X_valid)


In [ ]:
results = MLP_Search(alphas,activation,solver,h,maxiter,X_train_trans, X_Valid_trans, Y_train, Y_valid)
best_params_trans = FindBestScore(results)
# alpha 0.001 tanh adam learning rate = constant hidden layer 900 Inter 1000 score 0.65

In [ ]:
#Best score relu adam hidden layer = 300 Iter 1000 learning rate = constant score=0.66 
clf = MLPClassifier(hidden_layer_sizes=(best_params_trans["h"]), max_iter=maxiter, alpha=best_params_trans["alpha"],solver=best_params_trans["solver"],activation=best_params_trans["mode"],tol=1e-9,verbose=False)
clf.fit(X_trainN, Y_train)
Y_pred_trans = clf.predict(X_Valid_trans)
LC = clf.loss_curve_
Statistic(Y_valid,Y_pred_trans,"MLP trans")
plotLC(LC,best_params_trans)

In [2]:
df = pd.read_csv('Datasets/amazon/amazon_review_ID.shuf.lrn.csv')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df.drop('Class', axis=1))
scaled_features = scaler.transform(df.drop('Class', axis=1))
df_feat = pd.DataFrame(scaled_features, columns=df.columns[:-1])

df_test = pd.read_csv('Datasets/amazon/amazon_review_ID.shuf.tes.csv')
df_test_feat = pd.DataFrame(df_test, columns=df.columns[:-1])

X_train = df_feat
X_test = df_test_feat
X_test = scaler.transform(X_test)
y_train = df['Class']

quantile_transformer = QuantileTransformer(random_state=0)
X_train_trans = quantile_transformer.fit_transform(X_train)
X_test_trans = quantile_transformer.transform(X_test)


In [ ]:
X_test

In [3]:
clf = MLPClassifier(hidden_layer_sizes=(1000), max_iter=200, alpha=0.001,solver="adam",activation="relu",tol = 1e-9,verbose=True)
clf.fit(X_train_trans, y_train)
Y_pred = clf.predict(X_test_trans)
#accuracy_score(y_train, Y_pred)

Iteration 1, loss = 4.19922463
Iteration 2, loss = 1.94218581
Iteration 3, loss = 0.45591279
Iteration 4, loss = 0.11004849
Iteration 5, loss = 0.02774336
Iteration 6, loss = 0.01482367
Iteration 7, loss = 0.01023202
Iteration 8, loss = 0.00782130
Iteration 9, loss = 0.00699698
Iteration 10, loss = 0.00669165
Iteration 11, loss = 0.00653047
Iteration 12, loss = 0.00643187
Iteration 13, loss = 0.00637039
Iteration 14, loss = 0.00632517
Iteration 15, loss = 0.00629408
Iteration 16, loss = 0.00626947
Iteration 17, loss = 0.00625009
Iteration 18, loss = 0.00623413
Iteration 19, loss = 0.00621931
Iteration 20, loss = 0.00620634
Iteration 21, loss = 0.00619466
Iteration 22, loss = 0.00618326
Iteration 23, loss = 0.00617319
Iteration 24, loss = 0.00616315
Iteration 25, loss = 0.00615347
Iteration 26, loss = 0.00614419
Iteration 27, loss = 0.00613495
Iteration 28, loss = 0.00612585
Iteration 29, loss = 0.00611696
Iteration 30, loss = 0.00610835
Iteration 31, loss = 0.00609973
Iteration 32, los

In [1]:
from sklearn.ensemble import RandomForestClassifier as RFC

In [8]:
oob_scores = [0]
valid_scores = [0]
best_params = None
for n_ests in range(100,1000+1,100):
    print(n_ests)
    rf = RFC(n_jobs=-1, bootstrap=True, oob_score=True, n_estimators=n_ests, max_features="auto")#, max_depth=max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split,verbose=0)

    rf.fit(X_train, y_train)
   
    print("oob score:", rf.oob_score_)
    print("max depth:", max([estimator.get_depth() for estimator in rf.estimators_]))
    Y_pred = rf.predict(X_train)
    #score = accuracy_score(Y_valid, Y_pred)
    #print(f"number of validation samples: {X_valid.shape[0]}")
    #print(f"accuracy: {score*100:.2f}% --> {accuracy_score(Y_valid, Y_pred, normalize=False)}")
    #print(f"traintime: {traintime}s")
    print("-"*30)
    

100
oob score: 0.43866666666666665
max depth: 56
------------------------------
200
oob score: 0.532
max depth: 57
------------------------------
300
oob score: 0.5746666666666667
max depth: 66
------------------------------
400
oob score: 0.5973333333333334
max depth: 62
------------------------------
500
oob score: 0.6026666666666667
max depth: 64
------------------------------
600
oob score: 0.6386666666666667
max depth: 62
------------------------------
700
oob score: 0.6453333333333333
max depth: 66
------------------------------
800
oob score: 0.644
max depth: 65
------------------------------
900
oob score: 0.6706666666666666
max depth: 64
------------------------------
1000
oob score: 0.6733333333333333
max depth: 60
------------------------------


In [16]:
rf = RFC(n_jobs=-1, bootstrap=True, oob_score=True, n_estimators=1000, max_features="auto")
rf.fit(X_train, y_train)
Y_pred = rf.predict(X_train)

In [16]:
solution_data = list(zip(list(range(750,1500)), Y_pred))

In [17]:
solution_table = pd.DataFrame(solution_data, columns=['ID', 'Class'])
solution_table.head()

,ID,Class
0,750,Blankenship
1,751,Hayes
2,752,Goonan
3,753,Dent
4,754,Engineer


In [18]:
solution_table

,ID,Class
0,750,Blankenship
1,751,Hayes
2,752,Goonan
3,753,Dent
4,754,Engineer
...,...,...
745,1495,Corn
746,1496,Agresti
747,1497,Lawyeraau
748,1498,Grove


In [21]:
solution_table.to_csv(f"results/Amazon/SC_am_sol_QT.csv", index=False)

In [3]:
solution_table_ref = pd.read_csv('results/Amazon/SC_am_sol.csv')
solution_table = pd.read_csv('results/Amazon/SC_am_sol_1000x3.csv')

In [4]:
solution_table_ref

,ID,Class
0,750,Blankenship
1,751,Hayes
2,752,Goonan
3,753,Agresti
4,754,Engineer
...,...,...
745,1495,Cholette
746,1496,Agresti
747,1497,Janson
748,1498,Grove


In [10]:
tr = 0
fl = 0
for i in range(len(solution_table)):
    if (solution_table.Class[i] == solution_table_ref.Class[i]):
        #print(i,"True")
        tr = tr + 1
    else:
        #print(i,"False")
        fl = fl + 1
print(tr,fl)
tr/len(solution_table)

376 374


0.5013333333333333

In [ ]:
Class_description[int(solution_table.Class[0])]

In [ ]:
print(df.Class[750:1500])

In [ ]:
for i in range(0,len(solution_table.Class)):
    #df.Class[df.Class == Class_description[i]] = i+1
    #solution_table[["Class"]] = Class_description[int(solution_table.Class[i])]
    print(i,Class_description[int(solution_table.Class[i])])
    #print(Class_description[int(solution_table.Class[i])])